Dopo aver interpretato la query e pesato le feature, il sistema confronta il profilo richiesto con tutti i giocatori presenti nell’indice. La similarità viene calcolata combinando sia il punteggio IR (come la coseno-similarità dei vettori TF-IDF o lo score BM25) sia l’allineamento numerico delle statistiche pesate. Il risultato è un ranking finale dei giocatori più compatibili con il profilo cercato. Oltre a presentare la lista ordinata, il sistema può anche generare una breve spiegazione che giustifica perché un determinato giocatore si trova in cima alla classifica, evidenziando le caratteristiche maggiormente allineate alla query.

In [1]:
import os
import re
import numpy as np
import pandas as pd
import joblib

from sklearn.metrics.pairwise import cosine_similarity

# ============================
# 1. Caricamento indice + dataset stats
# ============================

MODELS_DIR = "../data/models"
DATA_CLEAN_PATH = "../data/drafted_cleaned.csv"

VECTORIZER_PATH = os.path.join(MODELS_DIR, "tfidf_vectorizer.joblib")
MATRIX_PATH = os.path.join(MODELS_DIR, "tfidf_matrix.joblib")
META_PATH = os.path.join(MODELS_DIR, "index_metadata.csv")

vectorizer = joblib.load(VECTORIZER_PATH)
tfidf_matrix = joblib.load(MATRIX_PATH)
metadata_df = pd.read_csv(META_PATH)

print("Indice caricato.")
print("TF-IDF matrix shape:", tfidf_matrix.shape)
print("Metadata shape:", metadata_df.shape)

# Carico dataset completo con le stats per spiegazioni
full_df = pd.read_csv(DATA_CLEAN_PATH)
full_df["player_id"] = full_df.index.astype(int)
print("Dataset stats shape:", full_df.shape)



if "player_id" not in metadata_df.columns:
    metadata_df["player_id"] = metadata_df.index.astype(int)

# ============================
# 2. Utils
# ============================

def normalize_text(text: str):
    if not isinstance(text, str):
        return ""
    return re.sub(r"\s+", " ", text.lower()).strip()

def safe_get(row, col, default=None):
    if col in row and not pd.isna(row[col]):
        return row[col]
    return default

def normalize_percentage(v):
    try:
        v = float(v)
    except:
        return None
    if v == 0:
        return 0
    if 0 < v < 0.8:
        return v * 100
    return v

# ============================
# 3. BUCKET UFFICIALI (TUOI)
# ============================

def bucketize_3p(pct):
    v = normalize_percentage(pct)
    if v is None: return None
    if v >= 40: return "ottimo tiratore da 3 punti"
    if v >= 35: return "buon tiratore da 3 punti"
    if v >= 30: return "tiratore da 3 punti discreto"
    return "tiratore da 3 punti poco affidabile"

def bucketize_ft(pct):
    v = normalize_percentage(pct)
    if v is None: return None
    if v <= 1: v *= 100
    if v >= 85: return "eccellente tiratore di tiri liberi"
    if v >= 75: return "buon tiratore di tiri liberi"
    return "tiratore di liberi migliorabile"

def bucketize_reb(reb):
    if reb is None: return None
    try: v = float(reb)
    except: return None
    if v >= 9: return "forte rimbalzista"
    if v >= 7: return "buon rimbalzista"
    if v >= 4: return "rimbalzista discreto"
    return "rimbalzista non di impatto"

def bucketize_ast(ast):
    if ast is None: return None
    try: v = float(ast)
    except: return None
    if v >= 9: return "ottimo playmaker e passatore"
    if v >= 6: return "buon passatore"
    if v >= 3: return "buon assistman"
    if v >= 1: return "assistman discreto"
    return "non particolarmente orientato agli assist"

def bucketize_scorer(pts):
    if pts is None: return None
    try: v = float(pts)
    except: return None
    if v >= 25: return "realizzatore di altissimo livello"
    if v >= 18: return "buon realizzatore"
    if v >= 12: return "realizzatore discreto"
    return "giocatore non focalizzato sulla realizzazione"

# ============================
# 4. Pattern skill + status
# ============================

SKILL_PATTERNS = {
    "shooting_3": ["tiratore da 3", "ottime percentuali da 3", "da 3 punti", "tripla"],
    "ft": ["tiri liberi", "liberi", "buon ft", "ottimo ft"],
    "reb": ["rimbalzista", "forte a rimbalzo", "rimbalzi"],
    "playmaking": ["assist", "playmaker", "passatore"],
    "scorer": ["tanti punti", "realizzatore", "prima opzione offensiva"]
}

STATUS_PATTERNS = {
    "Active": ["in attività", "ancora in attività", "gioca ancora"],
    "Retired": ["ritirato", "non gioca più", "ha smesso"]
}

SEMANTIC_EXPANSIONS = {
    "shooting_3": "ottimo tiratore da 3 3 punti 3P% buon tiratore da 3",
    "ft": "tiri liberi FT% buon tiratore ai liberi eccellente tiratore ai liberi",
    "reb": "rimbalzi forte rimbalzista buon rimbalzista",
    "playmaking": "assist buon passatore ottimo playmaker",
    "scorer": "segna molti punti realizzatore punti a partita"
}

SEMANTIC_WEIGHTS = {
    "shooting_3": 4,
    "ft": 3,
    "reb": 3,
    "playmaking": 3,
    "scorer": 3
}

# ============================
# 5. Parsing query -> intenti
# ============================

def parse_query_intents(q):
    q_norm = normalize_text(q)
    skills = set()
    status = None

    for s, pats in STATUS_PATTERNS.items():
        if any(p in q_norm for p in pats):
            status = s

    for skill, pats in SKILL_PATTERNS.items():
        if any(p in q_norm for p in pats):
            skills.add(skill)

    return {
        "normalized_query": q_norm,
        "skills": sorted(list(skills)),
        "status": status
    }

# ============================
# 6. Query pesata TF-IDF
# ============================

def build_weighted_query(raw_query, intents):
    base = intents["normalized_query"]
    boosted = [base]

    for skill in intents["skills"]:
        exp = SEMANTIC_EXPANSIONS.get(skill, "")
        w = SEMANTIC_WEIGHTS.get(skill, 1)
        boosted.append((" " + exp) * w)

    return " ".join(boosted).strip()

# ============================
# 7. Ricerca IR
# ============================

def search_players_scout(query, top_k=2000):
    intents = parse_query_intents(query)
    weighted_query = build_weighted_query(query, intents)

    q_vec = vectorizer.transform([weighted_query])
    sim = cosine_similarity(q_vec, tfidf_matrix).ravel()

    idx = np.argsort(sim)[::-1]
    results = metadata_df.iloc[idx].copy()
    results["similarity"] = sim[idx]

    results.attrs["intents"] = intents
    return results.head(top_k)

# ============================
# 8. Rank + Explain + FILTER AND
# ============================

def rank_and_explain(query, top_k=10):

    base = search_players_scout(query, top_k=2000)
    intents = base.attrs["intents"]

    merged = base.merge(full_df, on="player_id", how="left")
    

    # ====== Calcolo bucket ========
    merged["bucket_3p"] = merged["3P%"].apply(bucketize_3p)
    merged["bucket_ft"] = merged["FT%"].apply(bucketize_ft)
    merged["bucket_reb"] = merged["TRB.1"].apply(bucketize_reb)
    merged["bucket_ast"] = merged["AST.1"].apply(bucketize_ast)
    merged["bucket_pts"] = merged["PTS.1"].apply(bucketize_scorer)

    if intents["status"]:
        # Recupera il nome corretto della colonna Status dopo il merge
        status_col = None
        for c in merged.columns:
            if c.startswith("Status"):
                status_col = c
                break

        if status_col is None:
            raise ValueError("Nessuna colonna Status trovata dopo il merge.")

        merged = merged[merged[status_col] == intents["status"]]


    # ====== Filtro AND sulle skill ========
    required = intents["skills"]
    if required:
        conds = []

        for skill in required:

            if skill == "shooting_3":
                allowed = [
                "ottimo tiratore da 3 punti",
                "buon tiratore da 3 punti",
                "tiratore da 3 punti discreto",
                "tiratore da 3 punti poco affidabile"
                ]
                conds.append(merged["bucket_3p"].isin(allowed))

            if skill == "ft":
                allowed = [
                "eccellente tiratore di tiri liberi",
                "buon tiratore di tiri liberi",
                "tiratore di liberi migliorabile"
                ]
                conds.append(merged["bucket_ft"].isin(allowed))

            if skill == "reb":
                allowed = [
                "forte rimbalzista",
                "buon rimbalzista",
                "rimbalzista discreto",
                "rimbalzista non di impatto"
                ]
                conds.append(merged["bucket_reb"].isin(allowed))

            if skill == "playmaking":
                allowed = [
                "ottimo playmaker e passatore",
                "buon passatore",
                "assistman discreto",
                "non particolarmente orientato agli assist"
                ]
                conds.append(merged["bucket_ast"].isin(allowed))

            if skill == "scorer":
                allowed = [
                "realizzatore di altissimo livello",
                "buon realizzatore",
                "realizzatore discreto",
                "giocatore non focalizzato sulla realizzazione"
                ]
                conds.append(merged["bucket_pts"].isin(allowed))

        mask = conds[0]
        for c in conds[1:]:
            mask &= c

        merged = merged[mask]

    return merged.head(top_k), intents

# ============================
# 9. Test
# ============================

test_query = "Cerco un tiratore con ottime percentuali da 3, buone percentuali ai tiri liberi,forte passatore e che sia ritirato."
results, intents = rank_and_explain(test_query, top_k=10)

print("\nQuery originale:", test_query)
print("Intenti interpretati:", intents)
print("\nRisultati filtrati:")

# Scegli automaticamente la colonna Player corretta
if "Player_x" in results.columns:
    col_player = "Player_x"
elif "Player_y" in results.columns:
    col_player = "Player_y"
else:
    col_player = "Player"

cols = [col_player, "similarity"]

# Costruzione colonne dinamiche
cols = ["Player_x", "similarity"]

skill_to_bucket = {
    "shooting_3": "bucket_3p",
    "reb": "bucket_reb",
    "ft": "bucket_ft",
    "playmaking": "bucket_ast",
    "scorer": "bucket_pts",
}

# ================
# DISPLAY RISULTATI
# ================

def fmt_pct(x):
    if pd.isna(x): 
        return None
    try:
        v = float(x)
        if v < 1: v = v * 100
        return f"{v:.1f}%"
    except:
        return None

def fmt_num(x):
    if pd.isna(x): 
        return None
    try:
        return f"{float(x):.1f}"
    except:
        return None

# Costruisco colonne numeriche formattate
results["3P_value"]  = results["3P%"].apply(fmt_pct)
results["FT_value"]  = results["FT%"].apply(fmt_pct)
results["REB_value"] = results["TRB.1"].apply(fmt_num)
results["AST_value"] = results["AST.1"].apply(fmt_num)
results["PTS_value"] = results["PTS.1"].apply(fmt_num)


display_cols = [
    "Player_x",
    "similarity",
]

if "shooting_3" in intents["skills"]:
    display_cols += ["bucket_3p", "3P_value"]

if "ft" in intents["skills"]:
    display_cols += ["bucket_ft", "FT_value"]

if "reb" in intents["skills"]:
    display_cols += ["bucket_reb", "REB_value"]

if "playmaking" in intents["skills"]:
    display_cols += ["bucket_ast", "AST_value"]

if "scorer" in intents["skills"]:
    display_cols += ["bucket_pts", "PTS_value"]


display(results[display_cols])


Indice caricato.
TF-IDF matrix shape: (8323, 22019)
Metadata shape: (8323, 7)
Dataset stats shape: (8323, 37)

Query originale: Cerco un tiratore con ottime percentuali da 3, buone percentuali ai tiri liberi,forte passatore e che sia ritirato.
Intenti interpretati: {'normalized_query': 'cerco un tiratore con ottime percentuali da 3, buone percentuali ai tiri liberi,forte passatore e che sia ritirato.', 'skills': ['ft', 'playmaking', 'shooting_3'], 'status': 'Retired'}

Risultati filtrati:


,Player_x,similarity,bucket_3p,3P_value,bucket_ft,FT_value,bucket_ast,AST_value
0,Isiah Thomas,0.340140,tiratore da 3 punti poco affidabile,29.0%,buon tiratore di tiri liberi,75.9%,ottimo playmaker e passatore,9.3
1,Mark Price,0.334252,ottimo tiratore da 3 punti,40.2%,eccellente tiratore di tiri liberi,90.4%,buon passatore,6.7
2,Kevin Johnson,0.330443,tiratore da 3 punti discreto,30.5%,buon tiratore di tiri liberi,84.1%,ottimo playmaker e passatore,9.1
3,Magic Johnson,0.329624,tiratore da 3 punti discreto,30.3%,buon tiratore di tiri liberi,84.8%,ottimo playmaker e passatore,11.2
6,John Stockton,0.321873,buon tiratore da 3 punti,38.4%,buon tiratore di tiri liberi,82.6%,ottimo playmaker e passatore,10.5
8,Steve Nash,0.309092,ottimo tiratore da 3 punti,42.8%,eccellente tiratore di tiri liberi,90.4%,buon passatore,8.5
9,Scott Skiles,0.307085,buon tiratore da 3 punti,37.9%,eccellente tiratore di tiri liberi,88.9%,buon passatore,6.5
12,Oscar Robertson,0.292825,tiratore da 3 punti poco affidabile,-100.0%,buon tiratore di tiri liberi,83.8%,ottimo playmaker e passatore,9.5
13,Deron Williams,0.290939,buon tiratore da 3 punti,35.7%,buon tiratore di tiri liberi,82.2%,buon passatore,8.1
14,Damon Stoudamire,0.288207,buon tiratore da 3 punti,35.7%,buon tiratore di tiri liberi,83.3%,buon passatore,6.1
